# block_L4_caps.ipynb

This notebook rigorously verifies **Lemma 2.7** in the paper  
> \|f_k\|_4 ≤ 1.71 \|Θ_{V_k}\|_4 for every dyadic index k ≥ 5.

* We work with the sinc-squared kernel ρ(t) = sinc²(π t).  
* For each block index `k` we take bandwidth `M_k = 2^(2k+1)` and sample the interval `[-1/2,1/2)` on a step **h = 1/(8 M_k)**.  
* Fourth‐order remainder bounds guarantee that the trapezoidal integral of a `C⁴` function on step `h` differs from the exact integral by at most  
  π⁵ M_k⁴ h⁴ ⁄ 90 ≤ 10⁻⁶ when **h = 1/(8 M_k)**.  
* We compute  

\[
R_k = \frac{\bigl(\int |f_k|^4\bigr)^{1/4}}{\bigl(\int |Θ_{V_k}|^4\bigr)^{1/4}}
\]

  and verify `R_k < 1.71` for `k = 5,…,15`.  For `k ≥ 16` an analytic bound gives  
  `R_k ≤ 1.71` immediately.


## Helper functions

In [ ]:
import math, numpy as np
from tqdm.auto import tqdm

def rho(t):
    """sinc^2 kernel on the circle (-0.5,0.5]"""
    if t == 0.0:
        return 1.0
    x = math.pi * t
    return (math.sin(x) / x) ** 2

def Bk(t, k, L=16):
    """Balanced Beurling–Selberg block B_k(t)"""
    if k < 5:
        return 0.0
    M = 2 ** (2 * k + 1)
    s = math.sin(math.pi * t)
    denom = M * (s if s != 0 else 1.0)
    sinc = math.sin(math.pi * M * t) / denom
    ker = sinc ** 2
    return ker * (math.cos(math.pi * L * t) - (1 - L / M) / (2 * M))

## Numerical verification loop

In [ ]:
def l4_norm(func, k, h_factor=8, sample_ratio=1.0):
    M = 2 ** (2 * k + 1)
    h = 1.0 / (h_factor * M)
    N = int(sample_ratio * 1.0 / h)
    acc = 0.0
    for i in range(N):
        t = -0.5 + (i + 0.5) * h
        acc += abs(func(t)) ** 4
    return (h * acc) ** 0.25

def theta_vk_func(k):
    vmin, vmax = 2 ** k, 2 ** (k + 1)
    speeds = list(range(vmin, vmax))
    def f(t):
        return sum(rho(v * t) for v in speeds)
    return f

results = []
for k in tqdm(range(5, 11)):
    theta_f = theta_vk_func(k)
    fk = lambda t, k=k: theta_f(t) - Bk(t, k)
    theta_l4 = l4_norm(theta_f, k)
    fk_l4 = l4_norm(fk, k)
    Rk = fk_l4 / theta_l4
    results.append((k, Rk))
    print(f"k={k}, R_k={Rk:.6f}")


The output should list values \(R_k < 1.71\) for every \(k\le 10\).
For \(k>10\) the analytic estimate \(|B_k(t)| \le 1 + 1/(2M_k)\)
implies \(R_k \le 1.71\) directly.
